In [1]:
# 코어 스파크 라이브러리를 임포트
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName("Book Pipeline Project")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정
from IPython.display import display, display_pretty, clear_output, JSON
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size
spark

22/04/22 01:00:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Data Loading

In [2]:
book = spark.read.parquet("book_table/total/*.parquet")
book.printSchema()
# book.show(truncate=False)
display(book)

root
 |-- author: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- date: long (nullable = true)
 |-- description: string (nullable = true)
 |-- image: string (nullable = true)
 |-- ranking: integer (nullable = true)
 |-- title: string (nullable = true)



author,book_id,category,date,description,image,ranking,title
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.jpg?type=m5,36,행운 부르는 비즈


In [3]:
book.createOrReplaceTempView("book")
spark.sql("show tables 'book'")

database,tableName,isTemporary
,book,true


In [4]:
spark.sql("select count(*) as count from book")

count
4791


In [5]:
spark.sql("select category, count(*) as count from book group by category")

category,count
쉽게 읽는 과학,11
집/살림,39
천문/지구과학,32
한국사,35
불교,37
서양사,35
독서 에세이,37
연애/사랑 에세이,40
취미/레저,35
인간관계,23


## Data PreProcessing

In [6]:
!pip install konlpy # notebook 이미지 수정하기

     |████████████████████████████████| 19.4 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 9.0 MB/s eta 0:00:01


In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

#### 조사, 어미, 어간 등 제거

In [8]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

colsClean = udf(lambda z : clean(z), ArrayType(StringType()))
spark.udf.register("colsClean", colsClean)

def clean(s):
    okt = Okt()
    clean_words = []
    for word in okt.pos(s, stem=True):
        if word[1] in ['Noun', 'Verb', 'Adjective']:
            clean_words.append(word[0])
    new_doc = ' '.join(clean_words)
    return clean_words
            
tmp = book.withColumn('preprocessed', colsClean('description'))

In [9]:
tmp

author,book_id,category,date,description,image,ranking,title,preprocessed
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 알, 들어가다, 내용, 훑어보기, 작품, 분석, 하다, 작가, ..."
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구,"[분위기, 기능, 따르다, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 대다, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 하다, 도구, 재료, 설명, 비롯, 하다, 색다르다, 칠..."
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 하다, 이제, 수소, 에너지, 기존, 경제, 정치, 사회, 근본, 바꾸다, 것, 예견, 하다, ..."
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 만들다, 꿰, 방법, 쉬다, 이해, 하다, 평면도, 그리다, 놓다, 완성, 되다, 크기, 사용, 하다, 비즈, 낚싯줄, ..."
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리,"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 대해, 알아보다, 진주, 크리스털, 이용, 하다, 기본, 기..."
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션,"[비즈, 만들다, 법, 가지, 모으다, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 등, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 등, 대하, 일목요연하다, 기술, 있다..."
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈,"[세상, 단, 하나, 나, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 크다, 피어스, 목걸이, 핸드폰, 줄, 헤어, 핀, 등, 이용, 하다, ..."
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리,"[제작, 하다, 명품, 스타일, 비즈, 액세서리, 이, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 등, 액세서리, 포함, 하다, 핸드폰, 줄, 키홀, 더, 등, 제작, 하다,..."
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.jpg?type=m5,36,행운 부르는 비즈,"[탄생석, 목걸이, 귀고리, 가지, 이, 책, 비즈, 목걸이, 귀고리, 제작, 하다, 수, 있다, 구성, 되어다, 목걸이, 귀고리, 목걸이, 귀고리, 세트, 구별, 하다, 놓..."


#### 불용어 제거

In [17]:
import numpy
import pandas as pd
import nltk

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
df[0] = df[0].apply(lambda x: x.strip())
stopwords = df[0].to_numpy()

In [25]:
stopwords

array(['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나',
       '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하',
       '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받',
       '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르',
       '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나',
       '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명',
       '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻',
       '여자', '개', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘',
       '통하', '소리', '놓'], dtype=object)

In [37]:
# stopwords = spark.read.text("stopwords_kor.txt")
# stopwords.selectExpr("split(value, ' ')")
with open("stopwords_kor.txt", 'r') as f:
    stopwords = f.readlines()
stopwords = str(stopwords[0]).split(" ")
stopwords = numpy.array(stopwords)
stopwords

array(['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해',
       '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여',
       '입각하여', '기준으로', '예하면', '예를', '들면', '예를', '들자면', '저', '소인', '소생',
       '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수', '없다',
       '해서는', '안된다', '뿐만', '아니라', '만이', '아니다', '만은', '아니다', '막론하고',
       '관계없이', '그치지', '않다', '그러나', '그런데', '하지만', '든간에', '논하지', '않다',
       '따지지', '않다', '설사', '비록', '더라도', '아니면', '만', '못하다', '하는', '편이',
       '낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다',
       '오르다', '제외하고', '이', '외에', '이', '밖에', '하여야', '비로소', '한다면', '몰라도',
       '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할', '생각이다', '하려고하다',
       '이리하여', '그리하여', '그렇게', '함으로써', '하지만', '일때', '할때', '앞에서', '중에서',
       '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다',
       '할수있어', '임에', '틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만',
       '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼',
      

In [32]:
tmp.select('preprocessed')

preprocessed
"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 알, 들어가다, 내용, 훑어보기, 작품, 분석, 하다, 작가, ..."
"[분위기, 기능, 따르다, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 대다, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 하다, 도구, 재료, 설명, 비롯, 하다, 색다르다, 칠..."
"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 하다, 이제, 수소, 에너지, 기존, 경제, 정치, 사회, 근본, 바꾸다, 것, 예견, 하다, ..."
"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 만들다, 꿰, 방법, 쉬다, 이해, 하다, 평면도, 그리다, 놓다, 완성, 되다, 크기, 사용, 하다, 비즈, 낚싯줄, ..."
"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 대해, 알아보다, 진주, 크리스털, 이용, 하다, 기본, 기..."
"[비즈, 만들다, 법, 가지, 모으다, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 등, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 등, 대하, 일목요연하다, 기술, 있다..."
"[세상, 단, 하나, 나, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 크다, 피어스, 목걸이, 핸드폰, 줄, 헤어, 핀, 등, 이용, 하다, ..."
"[제작, 하다, 명품, 스타일, 비즈, 액세서리, 이, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 등, 액세서리, 포함, 하다, 핸드폰, 줄, 키홀, 더, 등, 제작, 하다,..."
"[탄생석, 목걸이, 귀고리, 가지, 이, 책, 비즈, 목걸이, 귀고리, 제작, 하다, 수, 있다, 구성, 되어다, 목걸이, 귀고리, 목걸이, 귀고리, 세트, 구별, 하다, 놓..."


In [38]:
nltk.download('punkt')

stopCols = udf(lambda x : cleanSW(x), StringType())
spark.udf.register("stopCols", stopCols)
def cleanSW(s):
    clean_words = []
#     for word in nltk.tokenize.word_tokenize(s):
    for word in s:
        if word not in stopwords:
            clean_words.append(word)
    return ' '.join(clean_words)

tmp2 = tmp.withColumn('final', stopCols('preprocessed'))

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
22/04/22 02:04:53 WARN SimpleFunctionRegistry: The function stopcols replaced a previously registered function.


In [35]:
tmp2

author,book_id,category,date,description,image,ranking,title,preprocessed,final
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 알, 들어가다, 내용, 훑어보기, 작품, 분석, 하다, 작가, ...",중학생 독후감 필 독선 시리즈 제 권 토마스 불 핀치 명작 그리스 로마 신화 작품 들어가다 내용 훑어보기 작품 분석 하다 작가 들여다보다 시대 연관 짓다 작품 토론 하다 독후...
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구,"[분위기, 기능, 따르다, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 대다, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]",분위기 기능 따르다 가구 선택 배치 주문 가구 정보 전국 대다 가구 시장 컬러 화보 위주 엮 인테리어 무크
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 하다, 도구, 재료, 설명, 비롯, 하다, 색다르다, 칠...",실내 벽 소품 가구 활용 가능하다 페인 팅 기법 컬러 사진 설명 생활 무크 페인팅 필요 하다 도구 재료 설명 비롯 하다 색다르다 칠하다 페인 팅기다 법리 폼 및 장식 아이디어...
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 하다, 이제, 수소, 에너지, 기존, 경제, 정치, 사회, 근본, 바꾸다, 것, 예견, 하다, ...",산업 시대 초기 석탄 증기 기관 새롭다 경제 패러다임 마련 하다 이제 수소 에너지 기존 경제 정치 근본 바꾸다 예견 하다 경제 노동 종말 소유 종말 저자 러미 리프킨 최신 작...
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 만들다, 꿰, 방법, 쉬다, 이해, 하다, 평면도, 그리다, 놓다, 완성, 되다, 크기, 사용, 하다, 비즈, 낚싯줄, ...",스 롭스 키 비즈 귀엽다 깜찍하다 동물 만들다 꿰 방법 쉬다 이해 하다 평면도 그리다 놓다 완성 되다 크기 사용 하다 비즈 낚싯줄 길이 공정 꿰 방법 세다 기록 하다 놓다 누...
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리,"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 대해, 알아보다, 진주, 크리스털, 이용, 하다, 기본, 기...",비즈 공예 입문 이자 전문 시작 마무리 규칙 기법 비즈 기본 종류 명칭 간단하다 도구 대해 알아보다 진주 크리스털 이용 하다 기본 기법 비즈 공예 하다 법 대해 진주 크리스털...
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션,"[비즈, 만들다, 법, 가지, 모으다, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 등, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 등, 대하, 일목요연하다, 기술, 있다...",비즈 만들다 법 모으다 엮었다 반지 목걸이 팔찌 귀걸이 다양하다 악세사리 제작 방법 비즈 종류 일목요연하다 기술 있다 부 재료 용법 게재 작품 제작 방법 대해 서도 상세 하다...
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈,"[세상, 단, 하나, 나, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 크다, 피어스, 목걸이, 핸드폰, 줄, 헤어, 핀, 등, 이용, 하다, ...",세상 단 보석 만들기 로맨틱 디자인 귀엽다 깜찍하다 디자인 반지 초 크다 피어스 목걸이 핸드폰 줄 헤어 핀 이용 하다 사랑스럽다 기품 있다 비즈 쥬얼리 만들다 있다 기본 아이...
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리,"[제작, 하다, 명품, 스타일, 비즈, 액세서리, 이, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 등, 액세서리, 포함, 하다, 핸드폰, 줄, 키홀, 더, 등, 제작, 하다,...",제작 하다 명품 스타일 비즈 액세서리 책 비즈 팔찌 발찌 반지 브로치 액세서리 포함 하다 핸드폰 줄 키홀 제작 하다 있다 구성 되어다 종류 별로 구별 하다 놓다 찾기 쉬다 해...
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.j

In [39]:
tmp2.select('final')

final
중학생 독후감 필 독선 시리즈 권 토마스 불 핀치 명작 그리스 로마 신화 작품 들어가다 내용 훑어보기 작품 분석 작가 들여다보다 시대 연관 짓다 작품 토론 독후감 예시 독후감...
분위기 기능 따르다 가구 선택 배치 주문 가구 정보 전국 대다 가구 시장 컬러 화보 위주 엮 인테리어 무크
실내 벽 소품 가구 활용 가능하다 페인 팅 기법 컬러 사진 설명 생활 무크 페인팅 필요 도구 재료 설명 비롯 색다르다 칠하다 페인 팅기다 법리 폼 장식 아이디어 소개
산업 시대 초기 석탄 증기 기관 새롭다 경제 패러다임 마련 이제 수소 에너지 기존 경제 정치 근본 바꾸다 예견 경제 노동 종말 소유 종말 저자 러미 리프킨 최신 작 작가 책 ...
스 롭스 키 비즈 귀엽다 깜찍하다 동물 만들다 꿰 방법 쉬다 이해 평면도 그리다 놓다 완성 크기 사용 비즈 낚싯줄 길이 공정 꿰 방법 세다 기록 놓다 쉬다 작품 만들다
비즈 공예 입문 이자 전문 시작 마무리 규칙 기법 비즈 기본 종류 명칭 간단하다 도구 알아보다 진주 크리스털 이용 기본 기법 비즈 공예 법 진주 크리스털 재료 매치 작품 만들...
비즈 만들다 법 모으다 엮었다 반지 목걸이 팔찌 귀걸이 다양하다 악세사리 제작 방법 비즈 종류 일목요연하다 기술 부 재료 용법 게재 작품 제작 방법 서도 상세 서술
세상 단 보석 만들기 로맨틱 디자인 귀엽다 깜찍하다 디자인 반지 초 크다 피어스 목걸이 핸드폰 헤어 핀 이용 사랑스럽다 기품 비즈 쥬얼리 만들다 기본 아이템 중심 다양하다 액...
제작 명품 스타일 비즈 액세서리 책 비즈 팔찌 발찌 반지 브로치 액세서리 포함 핸드폰 키홀 제작 구성 되어다 종류 별로 구별 놓다 찾기 쉬다 해 놓다 작품 난이도 제작 표시 ...
탄생석 목걸이 귀고리 책 비즈 목걸이 귀고리 제작 구성 되어다 목걸이 귀고리 목걸이 귀고리 세트 구별 놓다 찾기 쉬다 해 놓다 작품 난이도 제작 표시 관련 그림 사진 풍부하다 수록


In [ ]:
tmp2.select('preprocessed').collect()

#### 조사, 어간, 불용어 모두 제거

In [40]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import numpy

In [67]:
colsClean = udf(lambda z : clean(z), ArrayType(StringType()))
spark.udf.register("colsClean", colsClean)

def clean(s):
    okt = Okt()
    clean_words = []
    for word in okt.pos(s, stem=True):
        if word[1] in ['Noun', 'Verb', 'Adjective']:
            if word[0] not in stopwords:
                clean_words.append(word[0])
    return clean_words
   
with open("stopwords_kor.txt", 'r') as f:
    stopwords = f.readlines()
stopwords = str(stopwords[0]).split(" ")
stopwords = numpy.array(stopwords)
processed_book = book.withColumn('preprocessed', colsClean('description'))

22/04/23 09:35:17 WARN SimpleFunctionRegistry: The function colsclean replaced a previously registered function.


In [68]:
processed_book

author,book_id,category,date,description,image,ranking,title,preprocessed
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 들어가다, 내용, 훑어보기, 작품, 분석, 작가, 들여다보다, 시대,..."
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구,"[분위기, 기능, 따르다, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 대다, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 도구, 재료, 설명, 비롯, 색다르다, 칠하다, 페인, ..."
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제, 수소, 에너지, 기존, 경제, 정치, 근본, 바꾸다, 예견, 경제, 노동, 종말, 소유,..."
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 만들다, 꿰, 방법, 쉬다, 이해, 평면도, 그리다, 놓다, 완성, 크기, 사용, 비즈, 낚싯줄, 길이, 공정, 꿰, 방..."
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리,"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 알아보다, 진주, 크리스털, 이용, 기본, 기법, 비즈, 공..."
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션,"[비즈, 만들다, 법, 모으다, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 일목요연하다, 기술, 부, 재료, 용법, 게재, 작..."
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈,"[세상, 단, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 크다, 피어스, 목걸이, 핸드폰, 헤어, 핀, 이용, 사랑스럽다, 기품, 비즈, 쥬얼..."
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리,"[제작, 명품, 스타일, 비즈, 액세서리, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 액세서리, 포함, 핸드폰, 키홀, 제작, 구성, 되어다, 종류, 별로, 구별, 놓다, 찾..."
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.jpg?type=m5,36,행운 부르는 비즈,"[탄생석, 목걸이, 귀고리, 책, 비즈, 목걸이, 귀고리, 제작, 구성, 되어다, 목걸이, 귀고리, 목걸이, 귀고리, 세트, 구별, 놓다, 찾기, 쉬다, 해, 놓다, 작품, ..."


## 한계
불용어가 제대로 제거되지 않았다. (불용어 제거 전과 후의 차이가 별로 없는 것 확인)

한국어는 따로 정해진 불용어는 없고(nltk 한국어 불용어 리스트 없음) 토큰화 단계에서 조사.접속사를 제거하면 됨

필요없는 명사/형용사 제거하고 싶을 경우 직접 불용어 리스트 정의 후 한국어 불용어 제거하면 됨

## Modeling - Word2Vec

In [45]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import Word2VecModel

# inputCol=input Column name 
w2v = Word2Vec(vectorSize=100, seed=42, inputCol="preprocessed", outputCol="model")
model = w2v.fit(processed_book)
model.setInputCol("preprocessed")
model.getVectors().show()

22/04/23 07:23:40 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/23 07:23:40 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+------+--------------------+
|  word|              vector|
+------+--------------------+
|되어다|[0.14157755672931...|
|호흡기|[-0.0178339071571...|
|  근교|[0.03318461403250...|
|  사진|[-0.1910458207130...|
|  암시|[-0.0020179634448...|
|    스|[-0.0755680501461...|
|정재승|[-0.0371369421482...|
|파닉스|[0.35975518822669...|
|  마이|[-0.0909896194934...|
|신격호|[-0.0291027352213...|
|  권태|[-0.0436461940407...|
|  당국|[-0.0199533216655...|
|처지다|[-0.0562741532921...|
|십자가|[0.00179836456663...|
|  가드|[-0.0462693795561...|
|  한강|[0.00678041158244...|
|집주인|[-0.0311028826981...|
|  오기|[-0.0295059010386...|
|  관리|[-0.1665522903203...|
|  부사|[-0.0046996907331...|
+------+--------------------+
only showing top 20 rows



In [46]:
print(model.getVectorSize())
print(model.getVectors().first())

100
Row(word='되어다', vector=DenseVector([0.1416, -0.1855, 0.0128, 0.1544, -0.0692, -0.161, 0.112, 0.0796, 0.0164, 0.1209, -0.0454, -0.1715, -0.0782, 0.087, -0.2373, -0.1492, 0.2244, 0.0239, -0.0116, 0.0445, -0.0817, 0.0563, -0.072, 0.0617, 0.0488, -0.0626, -0.0186, 0.0382, 0.007, 0.3066, 0.1254, -0.0451, -0.0256, -0.1121, 0.1641, 0.0327, 0.0582, -0.0666, 0.1545, 0.0628, -0.0281, -0.1011, -0.076, -0.0542, 0.1039, 0.0312, -0.0848, -0.0952, -0.1522, 0.0395, 0.0347, -0.0062, -0.0424, -0.1733, 0.122, -0.1255, -0.1013, 0.0329, -0.0842, 0.1696, -0.1153, 0.2334, 0.1803, 0.07, -0.1411, 0.1331, -0.0246, -0.0425, -0.0826, -0.0242, -0.0332, 0.004, 0.1072, 0.1858, 0.0182, 0.1269, -0.0324, 0.2249, 0.1156, -0.0055, -0.0203, -0.0407, 0.0583, 0.0787, -0.2156, -0.2723, 0.1715, -0.0292, -0.0741, -0.0522, 0.1224, 0.1654, 0.1319, -0.1055, 0.0876, 0.1065, 0.0029, 0.1119, 0.0342, -0.1222]))


#### 결과 확인

In [53]:
model

Word2VecModel: uid=Word2Vec_d15e86781822, numWords=11359, vectorSize=100

In [47]:
from pyspark.sql.functions import format_number as fmt

model.findSynonyms("사진", 2).select("word", fmt("similarity", 5).alias("similarity"))

word,similarity
촬영,0.83778
카메라,0.77204


### 모델 저장

In [49]:
temp_path = "./"
# model metadata
w2vPath = temp_path + "/word2vec"
w2v.save(w2vPath)

# metadata, weight in parquet form
modelPath = temp_path + "/word2vec-model"
model.save(modelPath)

In [72]:
vecCols = udf(lambda x : get_document_vectors(x), DoubleType())
spark.udf.register("vecCols", vecCols)

def get_document_vectors(document_list):
    doc2vec = None; count = 0
    for word in document_list:
        if word in w2v_words:
            count += 1
            # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
            if doc2vec is None:
                doc2vec = word2vec_model[word]
            else:
                doc2vec = doc2vec + word2vec_model[word]

    if doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
        doc2vec = doc2vec / count

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return doc2vec

w2v_words = [row["word"] for row in model.getVectors().collect()]
vectorize_book = processed_book.withColumn('description_to_vec', get_document_vectors('preprocessed'))


22/04/23 09:43:55 WARN SimpleFunctionRegistry: The function veccols replaced a previously registered function.


AssertionError: col should be Column

In [71]:
processed_book.show()

+-----------------+-------+----------------+-------------+----------------------------------+--------------------+-------+-----------------------------------+------------------------------+
|           author|book_id|        category|         date|                       description|               image|ranking|                              title|                  preprocessed|
+-----------------+-------+----------------+-------------+----------------------------------+--------------------+-------+-----------------------------------+------------------------------+
|    토마스 불핀치|   2855|            신화|1648447114000| 중학생 독후감 필독선 시리즈 제...|https://bookthumb...|     34|     중학생이 보는 그리스 로마 신화| [중학생, 독후감, 필, 독선,...|
|효성출판사 편집부|  40565|        가구/DIY|1648449401000|분위기기능에 따른 가구선택과 배...|https://bookthumb...|     37|                  소가구 & 수납가구|[분위기, 기능, 따르다, 가구...|
|   중앙M&B 편집부|  66769|        가구/DIY|1648449401000| 실내 벽에서 소품가구까지 활용 ...|https://bookthumb...|     36|     내 손으로 더 예쁘게 페인팅 DIY|  [실내, 